In [1]:
!pip3 install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.0 MB 5.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import sqlalchemy
sqlalchemy.__version__

'2.0.19'

In [3]:
#create engine, which houses databases and DB connections
from sqlalchemy import create_engine
engine = create_engine("sqlite:///:memory:", echo=True)

In [4]:
#create the Base (declarative) to convert Python code to SQL code
from sqlalchemy.orm import declarative_base
Base = declarative_base()

In [5]:
#in Python, create a class, which will be converted by Alchemy to a SQL table
from sqlalchemy import Column, Integer, String
class User(Base):
  def __init__(self, name, fullname, nickname):
    __tablename__ = "users"
    self.id = Column(Integer, primary_key=True)
    self.name = Column(String)
    self.fullname = Column(String)
    self.nickname = Column(String)
    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name,
            self.fullname,
            self.nickname,
        )

InvalidRequestError: Class <class '__main__.User'> does not have a __table__ or __tablename__ specified and does not inherit from an existing table-mapped class.

In [ ]:
#confirm all the metadata in the User class
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [ ]:
#create all tables in the SQLlite database (in engine), commits and outputs SQL code
Base.metadata.create_all(engine)

2023-04-24 16:43:50,390 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-24 16:43:50,395 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("users")


2023-04-24 16:43:50,400 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)




2023-04-24 16:43:50,414 INFO sqlalchemy.engine.Engine [no key 0.00193s] ()


INFO:sqlalchemy.engine.Engine:[no key 0.00193s] ()


2023-04-24 16:43:50,416 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [ ]:
#create a User object
ed_user = User(name="ed", fullname="Ed Jones", nickname="edsnickname")
ed_user.name
ed_user.nickname
str(ed_user.name)

In [ ]:
#edit the object arguments
ed_user.nickname


'edsnickname'

In [ ]:
#a session allows us to insert/alter/delete data from the DB
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [ ]:
session = Session()

In [ ]:
#add the object to the DB via session
ed_user = User(name="ed", fullname="Ed Jones", nickname="edsnickname")
session.add(ed_user)

In [ ]:
#find the user object via query (queries the table and filters by column)
our_user= (
session.query(User).filter_by(name="ed").first()
)


2023-04-24 16:48:56,088 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-24 16:48:56,094 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)


2023-04-24 16:48:56,099 INFO sqlalchemy.engine.Engine [generated in 0.00511s] ('ed', 'Ed Jones', 'edsnickname')


INFO:sqlalchemy.engine.Engine:[generated in 0.00511s] ('ed', 'Ed Jones', 'edsnickname')


2023-04-24 16:48:56,106 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


2023-04-24 16:48:56,112 INFO sqlalchemy.engine.Engine [generated in 0.00560s] ('ed', 1, 0)


INFO:sqlalchemy.engine.Engine:[generated in 0.00560s] ('ed', 1, 0)


In [ ]:
#add multiple objects to the DB via session
session.add_all(
    [
        User(name="wendy", fullname="Wendy Williams", nickname="windy"),
        User(name="mary", fullname="Mary Contrary", nickname="mary"),
        User(name="fred", fullname="Fred Flintstone", nickname="freddy"),
    ]
)


In [ ]:
#example (do not run) of a programmatic way of adding numeric data
import numpy as np

session.add_all(
  for i in range (0,100):
    id = i
    code = np.sin(i*50/2)
    User(id = i, name = code, fullname= id+code, nickname = None))


In [ ]:
ed_user.nickname = "eddie"

In [ ]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [ ]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [ ]:
#commits all changes to the DB, outputs resulting SQL code as well
session.commit()

2023-04-24 16:51:55,109 INFO sqlalchemy.engine.Engine UPDATE users SET nickname=? WHERE users.id = ?


INFO:sqlalchemy.engine.Engine:UPDATE users SET nickname=? WHERE users.id = ?


2023-04-24 16:51:55,113 INFO sqlalchemy.engine.Engine [generated in 0.00443s] ('eddie', 1)


INFO:sqlalchemy.engine.Engine:[generated in 0.00443s] ('eddie', 1)


2023-04-24 16:51:55,117 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)


...

INFO:sqlalchemy.engine.Engine:INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)


2023-04-24 16:51:55,130 INFO sqlalchemy.engine.Engine [cached since 179s ago] ('fred', 'Fred Flintstone', 'freddy')


INFO:sqlalchemy.engine.Engine:[cached since 179s ago] ('fred', 'Fred Flintstone', 'freddy')


2023-04-24 16:51:55,133 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [ ]:
for user in session.query(User).order_by(User.id):
  print(user.name, user.fullname)

2023-04-24 16:53:40,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-24 16:53:40,353 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id


2023-04-24 16:53:40,358 INFO sqlalchemy.engine.Engine [generated in 0.00568s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00568s] ()


ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [ ]:
for u in session.query(User).order_by(User.id)[1:3]:
  print(u)


2023-04-24 17:06:41,375 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


2023-04-24 17:06:41,379 INFO sqlalchemy.engine.Engine [generated in 0.00411s] (2, 1)


INFO:sqlalchemy.engine.Engine:[generated in 0.00411s] (2, 1)


<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>


In [ ]:
for (name,) in session.query(User.name).filter(User.name=="Ed Jones"):
  print(name)

2023-04-24 17:17:35,699 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.name = ?


INFO:sqlalchemy.engine.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.name = ?


2023-04-24 17:17:35,716 INFO sqlalchemy.engine.Engine [generated in 0.01719s] ('Ed Jones',)


INFO:sqlalchemy.engine.Engine:[generated in 0.01719s] ('Ed Jones',)


In [ ]:
#alternatively, if you have an existing DB:
params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost\SQLEXPRESS;"
                                 "DATABASE=master;"
                                 "Trusted_Connection=yes")

## Connect using the specified parameters
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))


In [ ]:
## Query to pull data
timeSeries_query = """
SELECT *
FROM
	Sales
WHERE InvoiceDate BETWEEN '2010-12-01' AND '2011-01-01'
GROUP BY InvoiceDate, Description
ORDER BY InvoiceDate
"""
## Pull data using pandas
df_series = pd.read_sql(timeSeries_query, engine)
df_series['InvoiceDate'] = pd.to_datetime(df_series['InvoiceDate'])
